In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import math as mt
import random
import scipy.stats as st
import matplotlib.pyplot as plt

In [2]:
#Sets
T=10
J=3
periodos=list(t+1 for t in range(T))
bloques=list(j+1 for j in range(J))

#Parameters
A={(1):300000,(2):300000,(3):300000}
U=900000
S={(1):1.5,(2):3,(3):6}
C={(j+1):0.3*S[j+1] for j in range(J)}
H=300*S[1]
F=450*S[1]
R=60*S[1]
def qjt(a1,a2,a3,b1,b2,b3,j,t):
    if j==1:
        q_jt=max(-a1*(t-b1)**2+1,0)
    elif j==2:
        q_jt=max(-a2*(t-b2)**2+1,0)
    elif j==3:
        q_jt=max(-a3*(t-b3)**2+1,0)
    return q_jt
p={(1):900,(2):700,(3):500}

#Modelo
mdl=Model('Packing Grapes Harvest')

#Variables
x=mdl.addVars(bloques,periodos,vtype=GRB.CONTINUOUS,name='x')
y=mdl.addVars(periodos,vtype=GRB.INTEGER,name='y')
z=mdl.addVars(periodos,vtype=GRB.INTEGER,name='z')
m=mdl.addVars(periodos,vtype=GRB.INTEGER,name='m')
w=mdl.addVars(bloques,periodos,vtype=GRB.INTEGER,name='w')

#Objective
mdl.setObjective(quicksum(quicksum((S[j]*qjt(0.1,0.2,0.05,4,5,6,j=j,t=t)-C[j])*x[j,t] for j in bloques)-H*y[t]-F*z[t]-R*m[t] for t in periodos),sense=GRB.MAXIMIZE)

#Constrains
mdl.addConstrs(m[t]==m[t-1]+y[t]-z[t] if t>1 else m[t]==y[t]-z[t] for t in periodos)
mdl.addConstrs(quicksum(w[j,t] for j in bloques)<=m[t] for t in periodos)
mdl.addConstrs(p[j]*w[j,t]<=U for j in bloques for t in periodos)
mdl.addConstrs(x[j,t]<=p[j]*w[j,t] for j in bloques for t in periodos)
mdl.addConstrs(x[j,t]<=A[j]-quicksum(x[j,l] for l in range(1,t)) for j in bloques for t in periodos)

#Optimization
mdl.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-28
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 110 rows, 90 columns and 334 nonzeros
Model fingerprint: 0x43303a75
Variable types: 30 continuous, 60 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [3e-01, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+05, 9e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 78 rows and 47 columns
Presolve time: 0.00s
Presolved: 32 rows, 43 columns, 86 nonzeros
Variable types: 0 continuous, 43 integer (0 binary)

Root relaxation: objective 1.749214e+06, 19 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1749214.29    0   

In [3]:
print('Funcion Objetivo: '+str(mdl.ObjVal))
for j in bloques:
    for t in periodos:
        if x[j,t].X>0:
            print('x('+str(j)+','+str(t)+'): '+str(x[j,t].X))
        if w[j,t].X>0:
            print('w('+str(j)+','+str(t)+'): '+str(w[j,t].X))    
    print('\n')

print('\n')
for t in periodos:  
    if y[t].X>0:
        print('y('+str(t)+'): '+str(y[t].X))
    if z[t].X>0:    
        print('z('+str(t)+'): '+str(z[t].X))
    if m[t].X>0:    
        print('m('+str(t)+'): '+str(m[t].X))
    print('\n')


Funcion Objetivo: 1749015.0
x(1,3): 2100.0
w(1,3): 3.0
x(1,4): 297900.0
w(1,4): 331.0


x(2,4): 6300.0
w(2,4): 9.0
x(2,5): 238000.0
w(2,5): 340.0
x(2,6): 55700.0
w(2,6): 80.0


x(3,6): 130000.0
w(3,6): 260.0
x(3,7): 170000.0
w(3,7): 340.0








y(3): 3.0
m(3): 3.0


y(4): 337.0
m(4): 340.0


m(5): 340.0


m(6): 340.0


m(7): 340.0


m(8): 340.0


m(9): 340.0


m(10): 340.0


